Today, we will continue working on the inverted pendulum environment. However, the goal of this lab will be to implement one of the RL agents discussed in this week's lecture. Don't forget to download the new version of the environment under course contents. In the rest of this notebook, a Chainer skeleton code (with missing functionality) for the reinforce agent is provided. You can either work on finalizing the implementation of the reinforce agent or implement a different model of your choice.

In [6]:
import matplotlib.pyplot as plt
import tqdm
import numpy as np
from chainer import Chain
import chainer.links as L
import chainer.functions as F
from chainer.optimizers import Adam
from chainer import Variable
import random
import socket
import struct

%matplotlib inline

In [10]:
# same environment as last week

class Environment:
    def __init__(self, ip = "127.0.0.1", port = 13000):
        self.client = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
        self.ip     = ip
        self.port   = port

        self.client.connect((ip, port))

    def reset(self):
        self._send(0, 0)
        return self._receive()

    def step(self, action):
        self._send(action, 1)
        return self._receive()

    def _receive(self):
        data = self.client.recv(19)
        reward = data[0]
        state = [struct.unpack("@f", data[1 + i * 4: 5 + i * 4])[0] for i in range(4)]
        status = [data[17], data[18]]
        return reward, state, status

    def _send(self, action, command):
        self.client.send(bytes([action, command]))

In [11]:
# Let's define a baseline agent which just emits random actions.


class RandomAgent:
    def __init__(self):
        pass

    def step(self, reward, state):
        return random.randint(0, 1)

Let's run the agent within the environment. (don't forget to start the environment)
if you want to see the agent, also don't forget to enable the camera in the GUI

In [12]:
agent = RandomAgent()
environment = Environment()

In [ ]:
"""episode_count = 10
R0 = np.zeros(episode_count)

for i in range(1000):
    reward, state, status = environment.reset()

    while (status[0] == 0):
        action = agent.step(reward, state)
        reward, state, status = environment.step(action)
        R0[i] += reward
        """

In [13]:
# Let's create the REINFORCE agent. We assume that the policy is computed using an MLP with a softmax output.

class MLP(Chain):
    """Multilayer perceptron"""

    def __init__(self, n_output=1, n_hidden=5):
        super(MLP, self).__init__(l1=L.Linear(None, n_hidden), l2=L.Linear(n_hidden, n_output))

    def __call__(self, x):
        return self.l2(F.relu(self.l1(x)))

In [14]:
# A skeleton for the REINFORCEAgent is given. Implement the compute_loss and compute_score functions.

class REINFORCEAgent(object):
    """Agent trained using REINFORCE"""

    def __init__(self, model, optimizer=Adam()):
        self.model = model

        self.optimizer = optimizer
        self.optimizer.setup(self.model)

        # monitor score and reward
        self.rewards = []
        self.scores = []
        
        self.qvalues = []


    def step(self, reward, state):

        # linear outputs reflecting the log action probabilities and the value
        policy = self.model(Variable(np.atleast_2d(np.asarray(state, 'float32'))))

        # generate action according to policy
        p = F.softmax(policy).data

        # normalize p in case tiny floating precision problems occur
        row_sums = p.sum(axis=1)
        p /= row_sums[:, np.newaxis]

        action = np.asarray([np.random.choice(p.shape[1], None, True, p[0])])

        return action, policy


    def compute_loss(self):
        """
        Return loss for this episode based on computed scores and accumulated rewards
        """
    
        # cost to go
        Qhat = 0

        loss = 0
        for t in range(len(self.rewards) - 1, -1, -1):

            Qhat = self.rewards.pop() + Qhat

            _ss = F.squeeze(self.scores.pop(), axis=1) * Qhat

            if _ss.size > 1:
                _ss = F.sum(_ss, axis=0)
            loss += F.squeeze(_ss)

        return loss

    def compute_score(self, action, policy):
        """
        Computes score

        Args:
            action (int):
            policy:

        Returns:
            score
        """

        logp = F.log_softmax(policy)
    
        score = F.select_item(logp, Variable(action))

        # handle case where we have only one element per batch
        if score.ndim == 1:
            score = F.expand_dims(score, axis=1)

        return score

    def update_Q(self ):
        pass
        

In [15]:
# Now we run the REINFORCE agent within the environment. Note that we update the agent after each episode for simplicity.
# First, we should restart the server from the GUI

environment = Environment()
network = MLP(n_output=2, n_hidden=3)
agent = REINFORCEAgent(network, optimizer=Adam())

episode_count = 1000

    
R = np.zeros(episode_count)


In [19]:
for i in tqdm.trange(episode_count):

    reward, state, status = environment.reset()

    loss = 0
    while True:
        action, policy = agent.step(reward, state)
        reward, state, status = environment.step(action[0])
        print(state)
        # get reward associated with taking the previous action in the previous state
        agent.rewards.append(reward)
        R[i] += reward
        # recompute score function: grad_theta log pi_theta (s_t, a_t)
        agent.scores.append(agent.compute_score(action, policy))
        # we learn at the end of each episode
        if status[0] == 1:
            loss += agent.compute_loss()
            agent.model.cleargrads()
            loss.backward()
            loss.unchain_backward()
            agent.optimizer.update()
            agent.scores = []
            agent.rewards = []

            break



  0%|                                                                                         | 0/1000 [00:00<?, ?it/s]

[0.0, 0.0, 0.0, 0.0]
[0.0, 0.0, 0.0, 0.0]
[0.0, 0.0, 0.0, 0.0]
[-0.0, 0.0, 0.0, 0.0]
[-0.12457275390625, 0.020619487389922142, 0.0013689219485968351, 0.06844604015350342]
[-0.374481201171875, 0.0409686341881752, 0.004107620567083359, 0.136931911110878]
[-0.37701416015625, -0.0011573173105716705, 0.004107408691197634, -7.152557373046875e-07]
[-0.258453369140625, -0.022643402218818665, 0.0027404511347413063, -0.06834051012992859]
[-0.258453369140625, -0.022643402218818665, 0.0027404511347413063, -0.06834051012992859]
[-0.271759033203125, -0.0024529267102479935, 0.0027543306350708008, 0.00016768276691436768]
[-0.40057373046875, 0.017363248392939568, 0.004130084067583084, 0.06870739161968231]
[-0.405426025390625, -0.004511494189500809, 0.004116278141736984, -0.00015406310558319092]
[-0.29931640625, -0.02588786371052265, 0.0027445568703114986, -0.06850464642047882]
[-0.321197509765625, -0.0057448167353868484, 0.002758403541520238, 0.00015433132648468018]
[-0.321197509765625, -0.005744816735



  0%|                                                                                 | 1/1000 [00:01<21:06,  1.27s/it]

[0.0, 0.0, 0.0, 0.0]
[0.0, 0.0, 0.0, 0.0]
[-0.0, 0.0, 0.0, 0.0]
[-0.12457275390625, 0.020619487389922142, 0.0013689219485968351, 0.06844604015350342]
[-0.374481201171875, 0.0409686341881752, 0.004107620567083359, 0.136931911110878]
[-0.751495361328125, 0.060766249895095825, 0.008217941969633102, 0.2055051475763321]
[-1.258087158203125, 0.07973523437976837, 0.013702736236155033, 0.2742140293121338]
[-1.258087158203125, 0.07973523437976837, 0.013702736236155033, 0.2742140293121338]
[-1.897735595703125, 0.09760427474975586, 0.02056579850614071, 0.343107670545578]
[-2.674774169921875, 0.11411449313163757, 0.028811948373913765, 0.41223663091659546]
[-3.594390869140625, 0.12902438640594482, 0.0384470634162426, 0.4816531836986542]
[-4.662811279296875, 0.14211687445640564, 0.049478061497211456, 0.5514107942581177]
[-5.8870849609375, 0.15321233868598938, 0.061912983655929565, 0.6215644478797913]
[-5.8870849609375, 0.15321233868598938, 0.061912983655929565, 0.6215644478797913]
[-7.27529907226562



  0%|▏                                                                                | 2/1000 [00:01<18:17,  1.10s/it]

...



100%|██████████████████████████████████████████████████████████████████████████████▊| 998/1000 [15:47<00:01,  1.09it/s]

[0.0, 0.0, 0.0, 0.0]
[0.0, 0.0, 0.0, 0.0]
[-0.0, 0.0, 0.0, 0.0]
[-0.12457275390625, 0.020619487389922142, 0.0013689219485968351, 0.06844604015350342]
[-0.374481201171875, 0.0409686341881752, 0.004107620567083359, 0.136931911110878]
[-0.751495361328125, 0.060766249895095825, 0.008217941969633102, 0.2055051475763321]
[-0.751495361328125, 0.060766249895095825, 0.008217941969633102, 0.2055051475763321]
[-1.258087158203125, 0.07973523437976837, 0.013702736236155033, 0.2742140293121338]
[-1.897735595703125, 0.09760427474975586, 0.02056579850614071, 0.343107670545578]
[-2.674774169921875, 0.11411449313163757, 0.028811948373913765, 0.41223663091659546]
[-3.594390869140625, 0.12902438640594482, 0.0384470634162426, 0.4816531836986542]
[-4.662811279296875, 0.14211687445640564, 0.049478061497211456, 0.5514107942581177]
[-4.662811279296875, 0.14211687445640564, 0.049478061497211456, 0.5514107942581177]
[-5.8870849609375, 0.15321233868598938, 0.061912983655929565, 0.6215644478797913]
[-7.27529907226



100%|██████████████████████████████████████████████████████████████████████████████▉| 999/1000 [15:48<00:00,  1.15it/s]

[0.0, 0.0, 0.0, 0.0]
[0.0, 0.0, 0.0, 0.0]
[-0.0, 0.0, 0.0, 0.0]
[-0.0, 0.0, 0.0, 0.0]
[-0.12457275390625, 0.020619487389922142, 0.0013689219485968351, 0.06844604015350342]
[-0.118988037109375, -0.0005168095231056213, 0.0013550642179325223, -0.0001545250415802002]
[-0.249664306640625, 0.020009327679872513, 0.0027195829898118973, 0.06830665469169617]
[-0.501678466796875, 0.04007083922624588, 0.005456206388771534, 0.13682805001735687]
[-0.501678466796875, 0.04007083922624588, 0.005456206388771534, 0.13682805001735687]
[-0.881591796875, 0.059578265994787216, 0.009565195068717003, 0.2054375410079956]
[-1.391998291015625, 0.0782526358962059, 0.015049373731017113, 0.2741837501525879]
[-2.036346435546875, 0.09582437574863434, 0.02191263809800148, 0.34311652183532715]
[-2.819000244140625, 0.1120326891541481, 0.030159806832671165, 0.41228654980659485]
[-3.745208740234375, 0.12663748860359192, 0.03979680314660072, 0.4817464351654053]
[-4.821197509765625, 0.13942377269268036, 0.050830621272325516,



100%|██████████████████████████████████████████████████████████████████████████████| 1000/1000 [15:49<00:00,  1.17it/s]



In [ ]:
# and we finally plot the accumulated reward per episode

plt.figure()
plt.plot(np.cumsum(R0))
plt.plot(np.cumsum(R))
plt.legend(['Random', 'REINFORCE'])
plt.show()

In [ ]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:90% !important; }</style>"))